# Repositorio Scraping

# TODO

- Build paper catalog

- For each page in paper catalog, get links to papers
    
- get paper_data

In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import sqlite3
import time
import itertools
from contextlib import closing
import pandas as pd
import itertools

# Add to python path to import scripts
sys.path.append('../scripts/')

from src.sqlite import insert_urls, insert_html

## Create Tables

In [2]:
conn = sqlite3.connect("../saopaulo.db")
c = conn.cursor()

c.execute("DROP TABLE IF EXISTS catalog")
c.execute("""CREATE TABLE IF NOT EXISTS catalog (
          id INTEGER PRIMARY KEY,
          html TEXT,
          date_created TEXT,
          current INTEGER,
          parsed INTEGER
         )""")

c.execute("DROP TABLE IF EXISTS urls")
c.execute("""CREATE TABLE IF NOT EXISTS urls (
          id INTEGER PRIMARY KEY,
          items TEXT,
          url_stem TEXT UNIQUE,
          date_created TEXT,
          current INTEGER,
          url_scraped INTEGER
         )""")

conn.commit()
conn.close()

## Get Catalog HTMLs

In [12]:
# Helper functions
async def fetch_one(session, url, conn=None):
    async with session.get(url) as response:
        html_doc = await response.text()
    out_file = url.split("&page=")[1]
    with open(f"../output/{out_file}.html", "w") as f:
        f.write(html_doc)
        # insert_html(html_doc, date, conn)

async def fetch_all(urls, date, db="saopaulo.db"):
    # conn = sqlite3.connect(db)

    async with aiohttp.ClientSession() as session:
        tasks = (fetch_one(session, url) for url in urls)
        await asyncio.gather(*tasks)
        
    # conn.close()

        
# Execute
url_number = 300
date = '20220604'
url_root = 'https://repositorio.usp.br/result.php?filter%5B0%5D=unidadeUSP%3A%22EP%22&fields%5B0%5D=name&fields%5B1%5D=author.person.name&fields%5B2%5D=authorUSP.name&fields%5B3%5D=about&fields%5B4%5D=description&fields%5B5%5D=unidadeUSP&page='
urls = [url_root + str(i) for i in range(url_number)]
db = "../saopaulo.db"

t1 = time.perf_counter()
await fetch_all(urls, date, db=db)
t2 = time.perf_counter()
print(f"Took {t2-t1:.0f} seconds")

Took 37 seconds


In [9]:
urls[0].split("&page=")[1]

'0'

## Parse HTMLs and extract paper_urls

In [7]:
paper_urls = set()
for catalog_html in catalog_htmls:
    soup = BeautifulSoup(catalog_html, 'html5lib')
    articles = soup.find_all('article')
    urls = [article.p.a['href'] for article in articles]
    paper_urls.update(set(urls))

## Create papers table

In [92]:
with closing(sqlite3.connect("../saopaulo.db")) as conn:
    with conn:
        conn.execute(
            """
            CREATE TABLE IF NOT EXISTS papers (
                id INTEGER PRIMARY KEY,
                authors TEXT,
                orcids TEXT,
                author_urls TEXT,
                institution TEXT,
                date_created TEXT
            )
            """)

## Create edges table

In [79]:
with closing(sqlite3.connect("../saopaulo.db")) as conn:
    with conn:
        conn.execute(
            """
            CREATE TABLE IF NOT EXISTS edges (
                id INTEGER PRIMARY KEY,
                Source TEXT,
                Target TEXT,
                institution TEXT,
                date_created TEXT
            )
            """)

## Fetch HTMLs from paper_urls

In [18]:
# Fetch functions
async def fetch_author(session, url):
    async with session.get(url) as response:
        html = await response.text()
        return {'url':url, 'html':html}
    
async def fetch_all(urls):
    async with aiohttp.ClientSession() as session:
        tasks = (fetch_author(session, url) for url in urls)
        results = await asyncio.gather(*tasks)
        return results

In [55]:
# Fetch paper_urls from database
LIMIT = 10
conn = sqlite3.connect("../saopaulo.db")
with conn:
    # result = conn.execute("SELECT * FROM urls LIMIT (?)", (LIMIT,))
    result = conn.execute("SELECT * FROM urls WHERE id=9596")
    rows = result.fetchall()
conn.close()

url_root = 'https://repositorio.usp.br/'
urls = [url_root + row[2] for row in rows]

In [56]:
urls

['https://repositorio.usp.br/item/002742384']

In [20]:
# Fetch HTML
results = await fetch_all(urls)

## Parse paper_url HTML and get ORCID

In [22]:
for result in results:
    print(result['url'])

https://repositorio.usp.br/item/003040726
https://repositorio.usp.br/item/002902421
https://repositorio.usp.br/item/003036330
https://repositorio.usp.br/item/003033960
https://repositorio.usp.br/item/003040864
https://repositorio.usp.br/item/002861284
https://repositorio.usp.br/item/002784581
https://repositorio.usp.br/item/002765260
https://repositorio.usp.br/item/002781231
https://repositorio.usp.br/item/002774116


In [27]:
result = results[0]
html_doc = result['html']
soup = BeautifulSoup(html_doc, 'html5lib')

In [28]:
soup

<!DOCTYPE html>
<html dir="ltr" lang="pt-br"><head>
    <!-- Altmetric Script -->
    <script src="https://d1bxh8uas1mnw7.cloudfront.net/assets/embed.js" type="text/javascript"></script>

    <!-- PlumX Script -->
    <script crossorigin="anonymous" integrity="sha256-AXguJKYxgZY9FzwZE8U8EZxUQjcYT6iSQLLGiZTVW84=" src="https://cdn.plu.mx/widget-popup.js"></script>

  
    <link href="//repositorio.usp.br/inc/images/faviconUSP.ico" rel="shortcut icon" type="image/x-icon"/>

    <script src="//repositorio.usp.br/inc/js/jquery.min.js"></script>
    <script src="//repositorio.usp.br/inc/js/jquery-ui.js"></script>
    <link href="//repositorio.usp.br/inc/js/jquery-ui.css" rel="stylesheet"/>
    <script src="//repositorio.usp.br/inc/js/jquery.form-validator.min.js"></script>

    <!-- Uikit - Local -->
    <link href="//repositorio.usp.br/inc/uikit/css/uikit.min.css" rel="stylesheet"/>
    <script src="//repositorio.usp.br/inc/uikit/js/uikit.min.js"></script>
    <script src="//repositorio.usp

In [49]:
subjects

['ATUADORES PIEZELÉTRICOS',
 'CERÂMICA PIEZELÉTRICA',
 'ALGORITMOS',
 'ENGENHARIA MECÂNICA',
 'MÉTODO DOS ELEMENTOS FINITOS']

In [43]:
content[0].text

'Subjects: ATUADORES PIEZELÉTRICOS; CERÂMICA PIEZELÉTRICA; ALGORITMOS; ENGENHARIA MECÂNICA; MÉTODO DOS ELEMENTOS FINITOS'

In [91]:
# Check URL of first item
for result in results:
    # print(result['url'])
    # Convert to Beautiful Soup
    html_doc = result['html']
    soup = BeautifulSoup(html_doc, 'html5lib')
    
    # Extract ORCID and author url
    author_content = soup.select('article article ul li:-soup-contains("Authors: ") ul li')
    orcids = []
    author_names = []
    author_urls = []
    for list_elem in author_content:
        a_tags = list_elem.find_all('a')

        author_name = a_tags[0].text
        author_names.append(author_name)
        # print("name:", a_tags[0].text)

        author_url_raw = a_tags[0]['href'][2:]
        author_url = author_url_raw.split('/')[1].replace(" ", "%20").replace("\"", "%22")
        author_urls.append(author_url)
        # print("author url:", author_url)

        try:
            orcid = a_tags[1]['href'].split('/')[-1]
        except IndexError:
            orcid = ""
        # print("orcid:", orcid)
        orcids.append(orcid)
        # print()
    
    # Extract subjects
    subject_content = soup.select('article article ul li:-soup-contains("Subjects: ")')
    subjects = [a_tag.text for a_tag in subject_content[0].find_all('a')]

    url = result['url']
    
    with closing(sqlite3.connect("../saopaulo.db")) as conn:
        with conn:
            conn.execute(
            """
            INSERT INTO papers 
            ( url, authors, orcids, author_urls, subjects, institution )
            VALUES 
            (:url, :authors, :orcids, :author_urls, :subjects, :institution )
            """,
            (url, str(author_names), str(orcids), str(author_urls), str(subjects), 'EP')
            )

    nonempty_orcids = [x for x in orcids if x]
    if len(nonempty_orcids)<=1:
        return
    pairs = itertools.combinations(nonempty_orcids, 2)
    pairs = [pair + ('EP',) for pair in pairs]

    with closing(sqlite3.connect("../saopaulo.db")) as conn:
        with conn:
            conn.executemany(
            """
            INSERT INTO edges 
            ( Source, Target, institution )
            VALUES 
            (:Source, :Target, :institution)
            """,
            pairs
            )

## Output edges

In [38]:
with closing(sqlite3.connect("../saopaulo.db")) as conn:
    with conn:
        result = conn.execute(
        """
        SELECT * FROM edges LIMIT 10000;
        """)
        rows = result.fetchall()

df = pd.DataFrame(rows)
df = df[[1, 2]]
df.columns = ['Source', 'Target']
df.to_csv('../output/edges/edges_EP.csv', index=False)

## Get nodes

In [44]:
with closing(sqlite3.connect("../saopaulo.db")) as conn:
    print("Querying database")
    with conn:
        result = conn.execute(
        """
        SELECT orcids, authors, subjects FROM papers;
        """)

        rows = result.fetchall()
        
# Create orcids set
print("Creating set of orcids")
orcids_g = (eval(row[0]) for row in rows)
orcids = set(itertools.chain.from_iterable(orcids_g))
orcids.remove('')

new_rows = []
for row in rows:
    lst = [eval(lst) for lst in row]
    new_rows.append(lst)

t1 = time.perf_counter()
# Attach subject to each orcid key
print("Getting names and subjects")
nodes = {}
for orcid in orcids:
    for row in new_rows:
        if orcid in row[0]:
            # Get author name
            idx = row[0].index(orcid)
            name_lst = row[1]
            nodes[orcid] = [name_lst[idx]]

            # Get subjects
            nodes[orcid].append(row[2][0])

            break
t2 = time.perf_counter()
print(f"Done in {t2-t1:.0f} seconds.")

# Creating dataframe
df = pd.DataFrame.from_dict(nodes, orient='index').reset_index()
df.columns = ['id', 'label', 'subject']
# df.to_csv("output/nodes/nodes_EP.csv", index=False)

Querying database
Creating set of orcids
Getting names and subjects
Done in 5 seconds.


In [47]:
df.tail()

,id,label,subject
3166,0000-0002-8229-4027,"Dirani, Ely Antonio Tadeu",MATERIAIS ÓPTICOS
3167,0000-0003-4484-9199,"Afzalimir, Seyed Hamidreza",DUCTILIDADE
3168,0000-0002-4364-010X,"Cazzolato, Mirela Teixeira",BANCO DE DADOS
3169,0000-0002-5749-3198,"Pinheiro, Marina Julio - Universidade Estadual...",EQUILÍBRIO SÓLIDO-LÍQUIDO
3170,0000-0002-6694-0228,"Netto, Marcio Lobo (Orientador)",SISTEMAS MULTIAGENTES


In [45]:
len(df)

3171

# Debug (delete this)

In [ ]:
orcids = ['0000-0003-2977-6905', '', '0000-0002-6305-9662', '', '0000-0002-9723-1918']

# EXTRA CODE

In [1]:
import requests
from requests_html import HTMLSession

session = HTMLSession()

url = "https://repositorio.usp.br/result.php?filter[]=unidadeUSP:%22EP%22"
r = session.get(url)

In [43]:
r

<Response [200]>

In [44]:
# articles = r.text.select('article article')

In [45]:
# articles

In [6]:
authors = articles[0].find('p')[1]

In [7]:
authors

<Element 'p' class=('uk-article-meta', 'uk-margin-remove') style='color: #666'>

In [8]:
authors.html

'<p class="uk-article-meta uk-margin-remove" style="color: #666"><a class="link" href="result.php?filter[]=author.person.name:&quot;Oliveira, Christian J&#xFA;nior de&quot;">Oliveira, Christian Júnior de</a>; <a class="link" href="result.php?filter[]=author.person.name:&quot;Santos, Mois&#xE9;s Teles dos&quot;">Santos, Moisés Teles dos</a> <a href="https://orcid.org/0000-0002-8082-7619"><img src="inc/images/orcid_16x16.png"/></a>; <a class="link" href="result.php?filter[]=author.person.name:&quot;Vianna Junior, Ardson dos Santos&quot;">Vianna Junior, Ardson dos Santos</a> <a href="https://orcid.org/0000-0003-0192-7839"><img src="inc/images/orcid_16x16.png"/></a></p>'

In [9]:
authors.find('a')

[<Element 'a' class=('link',) href='result.php?filter[]=author.person.name:"Oliveira, Christian Júnior de"'>,
 <Element 'a' class=('link',) href='result.php?filter[]=author.person.name:"Santos, Moisés Teles dos"'>,
 <Element 'a' href='https://orcid.org/0000-0002-8082-7619'>,
 <Element 'a' class=('link',) href='result.php?filter[]=author.person.name:"Vianna Junior, Ardson dos Santos"'>,
 <Element 'a' href='https://orcid.org/0000-0003-0192-7839'>]

In [10]:
authors.find('a')[2].attrs['href']

'https://orcid.org/0000-0002-8082-7619'

In [11]:
for author in authors.find('a'):
    try:
        print(author.href)

SyntaxError: unexpected EOF while parsing (1236353434.py, line 3)